In [1]:
import pandas as pd

In [10]:
# format change taxonomic data (from .group to .csv) 
# for now only mpa4_class.profile is considered
taxonomy_df = pd.read_csv("data/mpa4_class.profile", sep="\t")
print(taxonomy_df.head())

# format change metadata (from .group to .csv)
metadata_df = pd.read_csv("data/sample.group", sep="\t", header=None)
print(metadata_df)
print(metadata_df.info())

                     name    850945    907005    907544    907995    910252  \
0           c__Clostridia  51.07453  19.25384  34.13202  40.45823  63.65824   
1          c__Bacteroidia  42.97830  67.00596  26.10095  49.82172  26.47992   
2        c__Negativicutes   2.08592   2.70182   0.41849   2.10271   3.52240   
3  c__Gammaproteobacteria   1.28249   9.25834  36.41831   0.00000   0.42619   
4   c__Betaproteobacteria   0.93501   0.95815   0.00000   3.79642   0.46030   

     917369    920071     A0002     A0005  ...  self_CRC23  self_CRC24  \
0  43.67910  48.12092  43.68023   6.24566  ...    53.93475    34.40354   
1  44.99695  30.36404  50.71171  88.15693  ...    26.12067     7.49449   
2   2.13157   8.24484   0.47678   0.81488  ...     6.59048     1.46674   
3   5.68301   0.00579   0.00000   0.20769  ...     1.93735     0.20606   
4   0.93574   1.52207   0.34718   1.96452  ...     0.04980     0.08714   

   self_CRC26  self_CRC28  self_CRC30  self_CRC31  self_CRC34  self_CRC35  \
0  

In [1]:
# rename and match metadata and taxonomic data for merging

taxonomy_df.rename(columns={taxonomy_df.columns[0]: "Taxa"}, inplace=True)
print(taxonomy_df.head())

metadata_df.columns = [
    "SampleID", "Group", "Group_1", "Group_2", "LevelB", "LevelA",
    "GroupDetails", "Project", "Project_1", "Project_2", "Unused_1",
    "Unused_2", "Unused_3", "Age", "Gender", "BMI", "Location",
    "LocationDetails", "SequenceType", "DNAExtractType"
]
print(metadata_df.head())

# verify match
print(set(taxonomy_df.columns[1:]).difference(metadata_df["SampleID"]))

NameError: name 'taxonomy_df' is not defined

In [3]:
# installing xgboost
pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 68.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 75.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# attempt to set up XGBoost without linking taxonomic data and metadata

import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Load the taxonomic dataset (mpa4_class.profile)
data = pd.read_csv("data/mpa4_class.profile", sep="\t")

# Transpose the dataset to have samples as rows and features as columns
data_transposed = data.set_index('name').transpose()
print(data_transposed.head())

name    c__Clostridia  c__Bacteroidia  c__Negativicutes  \
850945       51.07453        42.97830           2.08592   
907005       19.25384        67.00596           2.70182   
907544       34.13202        26.10095           0.41849   
907995       40.45823        49.82172           2.10271   
910252       63.65824        26.47992           3.52240   

name    c__Gammaproteobacteria  c__Betaproteobacteria  c__Bacilli  \
850945                 1.28249                0.93501     0.79336   
907005                 9.25834                0.95815     0.00000   
907544                36.41831                0.00000     2.22265   
907995                 0.00000                3.79642     0.10125   
910252                 0.42619                0.46030     3.11786   

name    c__CFGB1340  c__CFGB1424  c__Erysipelotrichia  \
850945      0.23746      0.18811              0.17254   
907005      0.00000      0.00000              0.04610   
907544      0.00000      0.00000              0.32344   
90

In [5]:

# Extract labels (y) - ensure you replace 'label_column' with your actual label column
labels = data_transposed.index.str.contains('self_CRC') 
y = labels.astype(int)  # Convert boolean to integer (1 for CRC, 0 for others)

# Drop the index to form the feature matrix (X)
X = data_transposed.reset_index(drop=True)
display(X)

name,c__Clostridia,c__Bacteroidia,c__Negativicutes,c__Gammaproteobacteria,c__Betaproteobacteria,c__Bacilli,c__CFGB1340,c__CFGB1424,c__Erysipelotrichia,c__Firmicutes_unclassified,...,c__CFGB1017,c__CFGB2546,c__Cytophagia,c__CFGB524,c__CFGB4422,c__Candidatus_Gracilibacteria_unclassified,c__CFGB497,c__CFGB1230,c__Acidobacteriia,c__Eurotiomycetes
0,51.07453,42.97830,2.08592,1.28249,0.93501,0.79336,0.23746,0.18811,0.17254,0.10786,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,19.25384,67.00596,2.70182,9.25834,0.95815,0.00000,0.00000,0.00000,0.04610,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34.13202,26.10095,0.41849,36.41831,0.00000,2.22265,0.00000,0.00000,0.32344,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,40.45823,49.82172,2.10271,0.00000,3.79642,0.10125,0.00000,0.00000,0.47637,0.21059,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,63.65824,26.47992,3.52240,0.42619,0.46030,3.11786,0.82004,0.00000,0.08038,0.31262,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8621,48.55523,23.72413,3.76489,5.16120,0.07380,2.20211,0.00000,0.00000,11.74268,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8622,23.39465,37.40127,1.03932,15.39485,0.04935,20.74146,0.00000,0.00000,0.03647,0.26414,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8623,47.10629,40.39927,1.21714,0.24941,0.00000,3.24044,0.00000,0.00000,1.53178,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8624,47.91039,34.96431,0.95754,0.13445,0.73767,1.37450,0.00000,0.00000,0.47432,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert to DMatrix format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set XGBoost parameters
params = {
    'objective': 'binary:logistic',  # Binary classification
    'eval_metric': 'logloss',       # Evaluation metric
    'max_depth': 6,                 # Maximum depth of trees
    'eta': 0.1,                     # Learning rate
    'verbosity': 1,                 # Verbosity level
    'seed': 42                      # Random seed for reproducibility
}

# Train the XGBoost model
num_round = 100
bst = xgb.train(params, dtrain, num_round)

# Predict on the test set
y_pred_prob = bst.predict(dtest)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

In [7]:

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(classification_report(y_test, y_pred))


Accuracy: 1.00
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1721
           1       0.00      0.00      0.00         5

    accuracy                           1.00      1726
   macro avg       0.50      0.50      0.50      1726
weighted avg       0.99      1.00      1.00      1726



/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
# what do the results tell us?
# the majority class is almost exclusively predicted, while the minority class is being ignored completely
# this might be due to a mismatch of samples/entries

In [12]:
# potential fixes:
# 1. reassembling dataset
# 2. adjusting weights 
# (...)